<h1 align="center">MACHINE LEARNING CLASSIFICATION MODEL FOR PCRF RESERVATIONS</h1>

After viewing and analyzing the data, we can (and should) create a classification Machine Learning model. We need to extract, clean, and process the data to find the best model for the classification job.

## IMPORTING LIBRARIES

We'll import the necessary libraries for preprocessing, creating and evaluating the Machine Learning classification model:

In [1]:
import os
from time import time
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import warnings

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
warnings.filterwarnings("ignore")

## EXTRACTING THE DATA

Let's read df_filtered file for construct our model:

In [4]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(project_dir, "data", "processed")
path = os.path.join(data_dir, 'df_filtered.csv')
df = pd.read_csv(path)

In [5]:
df.sample(3)

,index,facility_name,facility_type,center_longitude,center_latitude,site_name,permit_year,permit_month,event_start_year,event_start_month,event_start_time,day_of_week,attendance,hours_reserved,schedule_type,residency_flag,customer_gender,permit_hour,event_start_mounth_numeric,event_start_day_numeric,event_end_day_numeric,event_duration,day_of_week_numeric,permit_month_numeric
205276,205276,Rhodes Jr Baseball Field 1,Field - Baseball - School,-111.870846,33.380157,Schools,2014,December,2015,March,1900-01-01 18:00:00,Tuesday,25,4.00,Reservation: Non-Billable,True,Male,13,3,17,17,4.00,2,12
312026,312026,Group Fitness,Room - Fitness,-111.662972,33.432725,Recreation Centers,2015,April,2015,June,1900-01-01 11:00:00,Tuesday,15,0.75,Reservation: Internal,False,Female,15,6,30,30,0.75,2,4
4834,4834,Pickleball Court 01,Court - Pickleball,-111.741478,33.449963,Mesa Tennis & Pickleball Center,2022,January,2022,February,1900-01-01 16:30:00,Wednesday,4,1.00,Reservation: Billable,True,Female,16,2,2,2,1.00,3,1


In [6]:
df.shape

(434216, 24)

## DATA PREPROCESSING

For our model, we need to drop the columns that don't add value, reduce the outliers, transform the numeric variables to categorical, and reduce variable dimensions

### DROPING THE COLUMNS

Let's see the columns (again):

In [7]:
df.head(3)

,index,facility_name,facility_type,center_longitude,center_latitude,site_name,permit_year,permit_month,event_start_year,event_start_month,event_start_time,day_of_week,attendance,hours_reserved,schedule_type,residency_flag,customer_gender,permit_hour,event_start_mounth_numeric,event_start_day_numeric,event_end_day_numeric,event_duration,day_of_week_numeric,permit_month_numeric
0,0,Group Fitness,Room - Fitness,-111.662972,33.432725,Recreation Centers,2019,June,2022,January,1900-01-01 10:00:00,Thursday,1,1.00,Reservation: Internal,False,Mixed,14,1,27,27,1.00,4,6
1,1,Group Fitness,Room - Fitness,-111.662972,33.432725,Recreation Centers,2019,June,2022,January,1900-01-01 11:00:00,Thursday,1,0.75,Reservation: Internal,False,Mixed,14,1,27,27,0.75,4,6
2,2,Group Fitness,Room - Fitness,-111.662972,33.432725,Recreation Centers,2019,June,2022,January,1900-01-01 12:00:00,Thursday,1,0.75,Reservation: Internal,False,Mixed,14,1,27,27,0.75,4,6


The columns 'index', 'facility_name', 'site_name, 'permit_year', 'permit_month', 'permit_hour', 'event_start_time', 'event_start_year', 'event_start_month','event_end_time', 'event_start_day_numeric', 'event_end_day_numeric', 'day_of_week',and  'permit_month_numeric' don't add value to the model, so we can drop them:

In [8]:
df.drop(columns = ['index', 'facility_name','site_name', 'event_start_time', 'event_start_month', 'permit_month',
                    'event_start_day_numeric', 'event_end_day_numeric', 'day_of_week'], inplace=True)

### REDUCING OUTLIERS

In the EDA, we found that 'hours_reserved' has outliers that are greater than 22 hours. We can revisit this issue and remove these records (again)  :

In [9]:
df_2 = df[df['hours_reserved']<=20]
df_2.shape

(434197, 15)

In the same way, we can use the attendance filter extrating records smaller than 100:


In [10]:
df_3 = df_2[df_2['attendance']<=50]
df_3.shape

(373533, 15)

### TRANSFORMING IN NUMERIC VALUES

In order not to make the dataset bigger, we will use Label Encoding functionality:

In [11]:
le= LabelEncoder()

In [12]:
def label_encoder_function(dataframe, column):
    column_encoded = le.fit_transform(dataframe[column])
    return column_encoded

def add_nwe_column(dataframe, column):
    dataframe.loc[:,f'{column}_numeric']= label_encoder_function(dataframe, column)
    return dataframe

In [13]:
add_nwe_column(df_3, 'facility_type')
add_nwe_column(df_3, 'schedule_type')
add_nwe_column(df_3, 'residency_flag')
add_nwe_column(df_3, 'customer_gender')
df_3.shape

(373533, 19)

Now we must drop the old ones:

In [14]:
df_3 = df_3.drop(columns=['facility_type', 'schedule_type', 'residency_flag', 'customer_gender'])
df_3.shape

(373533, 15)

In [15]:
df_3.sample(3)

,center_longitude,center_latitude,permit_year,event_start_year,attendance,hours_reserved,permit_hour,event_start_mounth_numeric,event_duration,day_of_week_numeric,permit_month_numeric,facility_type_numeric,schedule_type_numeric,residency_flag_numeric,customer_gender_numeric
5254,-111.741478,33.449963,2022,2022,2,0.5,16,2,0.5,4,2,11,0,1,3
31123,-111.740991,33.477355,2022,2022,50,4.0,14,8,4.0,3,7,25,0,0,2
263939,-111.741478,33.449963,2017,2018,50,15.0,10,6,15.0,7,12,17,1,0,2


## GENERATING A PRONOSTIC MACHINE LEARNING MODEL

### FUNCTION TO PREPARE DATA

We'll create a function that standardizes the scale, divide data ofr train and test, trains models, splits the data for time series giving weights to the splits (giving more weight to the most recent splits), evaluates the models using these splits and selects (and save) the best model:

In [ ]:
def tscv_with_weighted_best_model(df, date_column, n_splits, target_column, models, n_iter, param_distributions):
    
    df = df.sort_values(by=date_column, ascending=True)
    
    
    train_size = int(len(df) * 0.7)
    train_df = df.iloc[:train_size]
    val_df = df.iloc[train_size:]
    
    
    X_train = train_df.drop(columns=[target_column])
    y_train = train_df[target_column]
    
    
    X_val = val_df.drop(columns=[target_column])
    y_val = val_df[target_column]
    
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    best_models = {}
    weights = np.linspace(0, 1, num=n_splits)  
    weighted_scores = {}
    start_time = time()
    print("Starting models training and evaluation with TimeSeriesSplit")
    
    for name, model in models.items():
        print(f"\nTraining and Evaluating {name}:")
        
        search = RandomizedSearchCV(model, param_distributions=param_distributions[name], 
                                  n_iter=n_iter, scoring='r2', cv=tscv, n_jobs=-1)
        search.fit(X_train, y_train)
        best_models[name] = search.best_estimator_
        print(f"Best params for {name}: {search.best_params_}")
        
       
        split_scores = []
        for i, (train_idx, test_idx) in enumerate(tscv.split(X_val)):
            X_val_train, X_val_test = X_val[train_idx], X_val[test_idx]
            y_val_train, y_val_test = y_val.iloc[train_idx], y_val.iloc[test_idx]
            best_model = best_models[name]  
            best_model.fit(X_val_train, y_val_train)
            preds = best_model.predict(X_val_test)
            score = r2_score(y_val_test, preds)
            split_scores.append(score * weights[i])
    
        weighted_scores[name] = np.sum(split_scores) / np.sum(weights)
        print(f"Weighted R² Score for {name}: {weighted_scores[name]:.4f}")
    
    
    final_model_name = max(weighted_scores, key=weighted_scores.get)
    final_model = best_models[final_model_name]
    
    end_time = time()
    print(f"\nFinal selected model: {final_model_name} with weighted R² Score: {weighted_scores[final_model_name]:.4f}")
    print(f"Total training time: {end_time - start_time:.2f} seconds")
    
    
    project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    model_dir = os.path.join(project_dir, "models")
    os.makedirs(model_dir, exist_ok=True)
    
    best_model_path = os.path.join(model_dir, "best_model.pkl")
    joblib.dump(final_model, best_model_path)
    print(f"✅ Model saved in: {best_model_path}")
    
    return final_model

We'll create a function that uses several machine learning regression models to find the best fit without presenting overfitting:

Now, let's define the models and their hyperparameter search spaces:

In [17]:
model_params = {
    'LinearRegression': {},

    'Ridge': {
        'alpha': uniform(0.1, 10),
        'solver': ['auto', 'svd', 'lsqr', 'sparse_cg', 'sag'],
        'max_iter': randint(100, 1000),
        'tol': uniform(1e-6, 1e-2),
    },

    'Lasso': {
        'alpha': uniform(0.1, 10),
        'tol': uniform(1e-6, 1e-2),
        'selection': ['cyclic', 'random']
    },

    'ElasticNet': {
        'alpha': uniform(0.1, 10),
        'l1_ratio': uniform(0, 1),
        'tol': uniform(1e-6, 1e-2),
        'selection': ['cyclic', 'random']
    },
    'RandomForestRegressor': {
        'n_estimators': randint(50, 500),
        'max_depth': randint(10, 100),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10),
        'max_features': ['sqrt', 'log2'],
    },

    'XGBRegressor': {
        'n_estimators': randint(50, 500),
        'learning_rate': uniform(0.01, 0.3),
        'max_depth': randint(3, 10),
        'min_child_weight': randint(1, 10),
        'gamma': uniform(0, 1),
        'subsample': uniform(0.1, 1),
        'colsample_bytree': uniform(0.1,0.8),
        'reg_alpha': uniform(0, 1),
        'reg_lambda': uniform(0, 1),
    }

}

models={'LinearRegression': LinearRegression(),
        'Ridge': Ridge(random_state=42),        
        'Lasso': Lasso(random_state=42),
        'ElasticNet': ElasticNet(random_state=42),
        'RandomForestRegressor': RandomForestRegressor(random_state=42),
        'XGBRegressor': XGBRegressor(tree_method='gpu_hist', device='cuda:0',random_state=42)
         }

Now, let's train and evaluate the models to find the best model. For training time series models, we must remove the first and last years to have complete data:

In [18]:
df_to_ml = df_3[(df_3["event_start_year"] > 2014) & (df_3["event_start_year"] < 2024)]
print(df_to_ml["event_start_year"].value_counts())

event_start_year
2023    57577
2022    55407
2021    52674
2019    35149
2015    32795
2020    31327
2016    30875
2017    28970
2018    27735
Name: count, dtype: int64


In [19]:
df_to_ml.shape

(352509, 15)

We have to use fewer iterations to find the best model using the created function:

In [20]:
n_iter = 2
n_splits = 5

date_column = 'event_start_year'
target_column = 'attendance'

print(f"Data size: {round(df_to_ml.shape[0])}")
print(f"Number of iterations: {n_iter}")

tscv_with_weighted_best_model(df_to_ml, date_column, n_splits, target_column, models,  n_iter, model_params)

Data size: 352509
Number of iterations: 2
Starting models training and evaluation with TimeSeriesSplit

Training and Evaluating LinearRegression:
Best params for LinearRegression: {}
Weighted R² Score for LinearRegression: -2505.7225

Training and Evaluating Ridge:
Best params for Ridge: {'alpha': 5.5192584758866845, 'max_iter': 482, 'solver': 'auto', 'tol': 0.005831290513035292}
Weighted R² Score for Ridge: 0.2482

Training and Evaluating Lasso:
Best params for Lasso: {'alpha': 2.713162566038807, 'selection': 'random', 'tol': 0.009068099745272395}
Weighted R² Score for Lasso: 0.1422

Training and Evaluating ElasticNet:
Best params for ElasticNet: {'alpha': 1.981815799698885, 'l1_ratio': 0.37138621243224035, 'selection': 'random', 'tol': 0.005332039465657486}
Weighted R² Score for ElasticNet: 0.1656

Training and Evaluating RandomForestRegressor:
Best params for RandomForestRegressor: {'max_depth': 78, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 12, 'n_estimator

RandomForestRegressor(max_depth=78, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=12, n_estimators=172, random_state=42)

The best model is **RandomForestRegressor**, so we can concentrate on it with more iterations:

In [21]:
model_params_2 = {

    'RandomForestRegressor': {
        'n_estimators': randint(50, 500),
        'max_depth': randint(10, 100),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10),
        'max_features': ['sqrt', 'log2'],
    },

}

models_2={
        'RandomForestRegressor': RandomForestRegressor(),
         }

In [23]:
n_iter = 100
n_splits = 9

date_column = 'event_start_year'
target_column = 'attendance'

print(f"Data size: {round(df_to_ml.shape[0])}")
print(f"Number of iterations: {n_iter}")

tscv_with_weighted_best_model(df_to_ml, date_column, n_splits, target_column, models_2,  n_iter, model_params_2)

Data size: 352509
Number of iterations: 100
Starting models training and evaluation with TimeSeriesSplit

Training and Evaluating RandomForestRegressor:
Best params for RandomForestRegressor: {'max_depth': 27, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 382}
Weighted R² Score for RandomForestRegressor: 0.8837

Final selected model: RandomForestRegressor with weighted R² Score: 0.8837
Total training time: 5306.99 seconds
✅ Model saved in: c:\Users\ingde\Documents\DAP\PRCF_Reservation\models\best_model.pkl


RandomForestRegressor(max_depth=27, max_features='sqrt', min_samples_split=4,
                      n_estimators=382)